In [ ]:
import pandas as pd
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8

df = pd.read_csv(
    '../input/santa-clara-county-covid19-vaccinations/COVID-19_vaccinations_among_county_residents_by_date.csv',
    index_col='Vaccine_Dt_rc',
    parse_dates=True
)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# What's the lag between "at least one shot" and "fully vaccinated"?

In [ ]:
df = df.assign(Total_vaccinated=lambda x: x.Cumulative_2nddose + x.Cumulative_singledose)
df['days_from_start'] = (df.index - df.index[0]).days
df = df.reset_index()
progress = (
    df[['Vaccine_Dt_rc','Cumulative_1stdose','Total_vaccinated']]
)

lag = (
    pd
    .merge_asof(
        progress,
        progress,
        left_on='Cumulative_1stdose',
        right_on='Total_vaccinated',
        #tolerance=5000,
        direction='forward' # Total_vaccinated >= Cumulative_1stdose
    )
    .assign(
        dose_delta=lambda x: x.Total_vaccinated_y - x.Cumulative_1stdose_x,
        time_delta=lambda x: x.Vaccine_Dt_rc_y - x.Vaccine_Dt_rc_x
    )
    [[
        'Vaccine_Dt_rc_x',
        'Cumulative_1stdose_x',
        'Vaccine_Dt_rc_y',
        'Total_vaccinated_y',
        'dose_delta',
        'time_delta'
    ]]
)

lag.describe()

In [ ]:
lag.plot(x='Vaccine_Dt_rc_x', y='time_delta')

In [ ]:
lag